In [1]:
from math import *
from scipy.interpolate import interp2d
import pandas as pd
import numpy as np

In [2]:
Kz_g = pd.read_excel('Коэф исп заземли.xlsx',sheet_name='Кг',index_col='n')
XYZ_g = Kz_g.unstack().reset_index()
XYZ_g.columns = ['a/L', 'n', 'K']
interpol_Kg = interp2d(x=XYZ_g['a/L'], y=XYZ_g['n'], z=XYZ_g['K'])

C:\Users\Vadim\miniconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=1,1 nx,ny=6,9 m=27 fp=0.000182 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


In [3]:
Kz_v = pd.read_excel('Коэф исп заземли.xlsx',sheet_name='Кв',index_col='n')
XYZ_v = Kz_v.unstack().reset_index()
XYZ_v.columns = ['a/L', 'n', 'K']
interpol_Kv = interp2d(x=XYZ_v['a/L'], y=XYZ_v['n'], z=XYZ_v['K'])

C:\Users\Vadim\miniconda3\lib\site-packages\scipy\interpolate\_fitpack_impl.py:977: RuntimeWarning: No more knots can be added because the number of B-spline
coefficients already exceeds the number of data points m.
Probable causes: either s or m too small. (fp>s)
	kx,ky=1,1 nx,ny=6,8 m=21 fp=0.000029 s=0.000000
  warnings.warn(RuntimeWarning(_iermess2[ierm][0] + _mess))


Произвести расчет заземляющего устройства для электроустановок напряжением 400 В. 

Грунт – суглинок. 

Стержни можно разместить по контуру КТП, имеющего в плане размеры 3 х 3 м.

Глубина заложения стержней от поверхности земли Н = 0,7 м.

1. Примем в качестве заземлителей стальные уголки длиной lc = 2.5 м 50х50х4 мм. Соединение заземлителей произведем на сварке стальной полосой шириной b = 40 мм.

In [84]:
h = 0.7
l = 3
d = 0.05
b = 0.04
h1 = h + l / 2
hg = 0.7

2. Удельное сопротивление грунта с учетом сезонных колебаний влажности для вертикальных стержней, Ом*м:

In [85]:
r_0 = 100

3. Сопротивление растеканию тока одиночного стержня, Ом

In [86]:
Rv = 0.366 * r_0 / l * (log10(2.1 * l / d) + 0.5 * log10((4 * h1 + l)/(4 * h1 - l)))
print(f'Rв = {Rv:.1f} Ом')

Rв = 27.5 Ом


5. Теоретическое число вертикальных заземлителей

In [99]:
nt = ceil(Rv / 4)
print(f'n_теор = {nt} шт')
nt = 10

n_теор = 7 шт


4. Сопротивление горизонтального заземлителя

In [115]:
L = (nt) * l * 1
print(L)

30


In [116]:
Rg = 0.366 * r_0 / L * log10((2 * L**2) / (b * hg))
print(f'Rг = {Rg:.1f} Ом')

Rг = 5.9 Ом


In [117]:
a = L / (nt)
a_L = a / l
print(f'a = {a}')
print(f'a/L = {a_L}')

a = 3.0
a/L = 1.0


принимаем nг и nв по Таблице:

In [118]:
ng = interpol_Kg(round(a_L), nt)[0]
#ng = 0.4
nv = interpol_Kv(round(a_L), nt)[0]
print(f'n_г = {ng:.2f}')
print(f'n_в = {nv:.2f}')

n_г = 0.34
n_в = 0.55


Результирующее сопротивление:

In [119]:
Rz = (Rv * Rg) / (Rv * ng + Rg * nv * nt)
print(f'Rз = {Rz:.1f} Ом')

Rз = 3.9 Ом
